## Get Insider Trading Details from NSE website
https://www.nseindia.com/companies-listing/corporate-filings-insider-trading

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import requests
from datetime import datetime
from datetime import timedelta  
import json
import time

### Get date range

In [2]:
start_time = time.time()
fromdate = datetime.strftime(datetime.today(),'%d-%m-%Y')
todate =  datetime.today() - timedelta(days=90)  # Calculate the date 90 days ago from today
enddate = datetime.strftime(todate,'%d-%m-%Y')

### Set up headers for the HTTP requests to mimic a browser and avoid being blocked


In [3]:
head = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
    "Upgrade-Insecure-Requests": "1",
    "DNT": "1",  # Do Not Track request header
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
}

### URL for the initial request to obtain necessary cookies and bypass anti-scraping measures
 Print the URL from which the data is fetched

In [4]:
URL1 = "https://www.nseindia.com/companies-listing/corporate-filings-insider-trading"
d1 = requests.get(URL1,headers=head) # Send the request and store the response

# URL for fetching insider trading data from NSE's API, using the date range calculated above
URL = 'https://www.nseindia.com/api/corporates-pit?index=equities&from_date='+ enddate+ '&to_date=' + fromdate
print(URL)

https://www.nseindia.com/api/corporates-pit?index=equities&from_date=17-05-2024&to_date=15-08-2024


In [5]:
# Send a GET request to the API with the necessary headers and cookies, then parse the response as JSON
d = requests.get(URL,headers=head,cookies =d1.cookies ).json()

# Convert the JSON response to a pandas DataFrame
df = pd.DataFrame(d['data'])
df


,symbol,company,anex,acqName,date,pid,tkdAcqm,buyValue,sellValue,buyQuantity,sellquantity,secType,secAcq,did,tdpTransactionType,tdpDerivativeContractType,xbrl,personCategory,befAcqSharesNo,befAcqSharesPer,secVal,securitiesTypePost,afterAcqSharesNo,afterAcqSharesPer,acqfromDt,acqtoDt,intimDt,acqMode,derivativeType,exchange,remarks
0,SWSOLAR,Sterling and Wilson Renewable Energy Limited,7(2),Kainaz Khurshed Daruvala,14-Aug-2024 22:10,1168635,None,0,0,0,0,Equity Shares,34818,513951,Pledge,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,13000200,5.57,23366360,Equity Shares,13000200,5.57,12-Aug-2024,12-Aug-2024,14-Aug-2024,Pledge Creation,-,NA,-
1,NIITMTS,Niit Learning Systems Limited,7(2),Bimal K Jain,14-Aug-2024 20:43,1168633,None,0,0,0,0,Equity Shares,9400,513944,Sell,-,https://nsearchives.nseindia.com/corporate/xbr...,-,54400,0.04,4426821,Equity Shares,45000,0.03,09-Aug-2024,09-Aug-2024,12-Aug-2024,Market Sale,-,NSE,-
2,NIITMTS,Niit Learning Systems Limited,7(2),Gaurav Relhan,14-Aug-2024 20:43,1168633,None,0,0,0,0,Equity Shares,10000,513946,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,-,Nil,0,1770900,Equity Shares,10000,0.01,12-Aug-2024,12-Aug-2024,12-Aug-2024,ESOP,-,NA,-
3,NIITMTS,Niit Learning Systems Limited,7(2),Bimal K Jain,14-Aug-2024 20:43,1168633,None,0,0,0,0,Equity Shares,15000,513945,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,-,45000,0.03,3020400,Equity Shares,60000,0.04,12-Aug-2024,12-Aug-2024,12-Aug-2024,ESOP,-,NA,-
4,NIITMTS,Niit Learning Systems Limited,7(2),Brandon David Dickens,14-Aug-2024 20:43,1168633,None,0,0,0,0,Equity Shares,6666,513948,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,-,Nil,0,352231,Equity Shares,6666,0,29-Jul-2024,29-Jul-2024,13-Aug-2024,ESOP,-,NA,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,TEJASNET,Tejas Networks Limited,7(2),Rupinder Singh,17-May-2024 00:15,1164145,None,0,0,0,0,Equity Shares,2000,504752,Sell,-,https://nsearchives.nseindia.com/corporate/xbr...,Employees/Designated Employees,65356,0.04,2267641,Equity Shares,63356,0.04,30-Apr-2024,30-Apr-2024,06-May-2024,Market Sale,-,NSE,-
4236,TEJASNET,Tejas Networks Limited,7(2),Sunil Radhakrishnan,17-May-2024 00:15,1164145,None,0,0,0,0,Equity Shares,10000,504751,Sell,-,https://nsearchives.nseindia.com/corporate/xbr...,Employees/Designated Employees,34735,0.02,11098454,Equity Shares,24735,0.01,07-May-2024,07-May-2024,09-May-2024,Market Sale,-,NSE,-
4237,TEJASNET,Tejas Networks Limited,7(2),Neelagandan P G,17-May-2024 00:15,1164145,None,0,0,0,0,Equity Shares,10500,504750,Sell,-,https://nsearchives.nseindia.com/corporate/xbr...,Employees/Designated Employees,41000,0.02,12313725,Equity Shares,30500,0.02,10-May-2024,13-May-2024,14-May-2024,Market Sale,-,BSE,-
4238,TEJASNET,Tejas Networks Limited,7(2),Manish Khandelwal,17-May-2024 00:15,1164145,None,0,0,0,0,Equity Shares,5000,504757,Sell,-,https://nsearchives.nseindia.com/corporate/xbr...,Employees/Designated Employees,18522,0.01,5425226,Equity Shares,13522,0.01,25-Apr-2024,07-May-2024,09-May-2024,Market Sale,-,NSE,-


### Cleaning the data
The code below replaces any '-' values in the 'secVal' column with 0, converts the string value to numeric, and converts the 'secAcq' column to numeric.

In [6]:
# Replace any '-' values in the 'secVal' column with 0
df["secVal"].replace({"-": 0}, inplace=True)

# Convert the string value to numeric
df["secVal"] = pd.to_numeric(df["secVal"])
df["secAcq"] = pd.to_numeric(df["secAcq"])

### Categorize Insider Trading for Promoters and acquision from Market Purchase

In [7]:
# catergorize only the promoters
personCat = ['Promoters','Promoter Group']

# Filter the DataFrame to only include rows where 'acqMode' is 'Market Purchase'
df = df[df['acqMode'] == 'Market Purchase']

# Filter the DataFrame to only include rows where 'secType' is 'Equity Shares'
df=df[df['secType']=='Equity Shares']

# Further filter the DataFrame to only include rows where 'personCategory' is in the 'personCat' list
df = df[df['personCategory'].isin(personCat) ]

# Convert the 'acqfromDt' column to a datetime format, specifying the format to match the date string
df['acqfromDt'] = pd.to_datetime(df['acqfromDt'], format='%d-%b-%Y',errors='ignore')

df

,symbol,company,anex,acqName,date,pid,tkdAcqm,buyValue,sellValue,buyQuantity,sellquantity,secType,secAcq,did,tdpTransactionType,tdpDerivativeContractType,xbrl,personCategory,befAcqSharesNo,befAcqSharesPer,secVal,securitiesTypePost,afterAcqSharesNo,afterAcqSharesPer,acqfromDt,acqtoDt,intimDt,acqMode,derivativeType,exchange,remarks
104,20MICRONS,20 Microns Limited,7(2),IONIX ADVANCED MATERIALS PRIVATE LIMITED,14-Aug-2024 16:49,1168575,None,0,0,0,0,Equity Shares,15943.0,513739,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,8633338,24.47,5074497,Equity Shares,8649281,24.51,2024-08-12,12-Aug-2024,12-Aug-2024,Market Purchase,-,NSE,-
118,STEL,Stel Holdings Limited,7(2),Digidrive Distributors Limited,14-Aug-2024 12:05,1168551,None,0,0,0,0,Equity Shares,21323.0,513709,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,160277,0.87,10336546,Equity Shares,181600,0.98,2024-08-12,12-Aug-2024,13-Aug-2024,Market Purchase,-,NSE,-
120,MANINFRA,Man Infraconstruction Limited,7(2),Mansi P. Shah,14-Aug-2024 09:35,1168546,None,0,0,0,0,Equity Shares,25000.0,513705,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoters,59396369,16,4715000,Equity Shares,59421369,16.01,2024-08-13,13-Aug-2024,14-Aug-2024,Market Purchase,-,NSE,-
159,UNIHEALTH,Unihealth Consultancy Limited,7(2),Mrs. Mayuri Akshay Parmar,13-Aug-2024 17:45,1168513,None,0,0,0,0,Equity Shares,2000.0,513622,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,94800,0.61,251950,Equity Shares,96800,0.63,2024-08-13,13-Aug-2024,13-Aug-2024,Market Purchase,-,NSE,-
160,UNIHEALTH,Unihealth Consultancy Limited,7(2),Mrs. Mayuri Akshay Parmar,13-Aug-2024 17:45,1168513,None,0,0,0,0,Equity Shares,1000.0,513621,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,93800,0.61,129000,Equity Shares,94800,0.61,2024-08-12,12-Aug-2024,13-Aug-2024,Market Purchase,-,NSE,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141,KIRLPNU,Kirloskar Pneumatic Company Limited,7(2),Kirloskar Systems Private Limited,21-May-2024 18:37,1164270,None,0,0,0,0,Equity Shares,2010.0,505019,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,Nil,0,2490691,Equity Shares,2010,0,2024-05-18,18-May-2024,20-May-2024,Market Purchase,-,NSE,-
4146,BHARATWIRE,Bharat Wire Ropes Limited,7(2),Murarilal Mittal,21-May-2024 16:09,1164258,None,0,0,0,0,Equity Shares,10000.0,505008,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoters,1317737,1.94,2720000,Equity Shares,1327737,1.95,2024-05-18,18-May-2024,18-May-2024,Market Purchase,-,NSE,-
4147,BHARATWIRE,Bharat Wire Ropes Limited,7(2),Gyanshankar E-Trading LLP,21-May-2024 16:09,1164258,None,0,0,0,0,Equity Shares,681.0,505007,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,442229,0.64,182849,Equity Shares,442910,0.65,2024-05-18,18-May-2024,18-May-2024,Market Purchase,-,NSE,-
4148,BHARATWIRE,Bharat Wire Ropes Limited,7(2),Gyanshankar E-Trading LLP,21-May-2024 16:09,1164258,None,0,0,0,0,Equity Shares,64400.0,505006,Buy,-,https://nsearchives.nseindia.com/corporate/xbr...,Promoter Group,377829,0.56,17498403,Equity Shares,442229,0.64,2024-05-15,17-May-2024,17-May-2024,Market Purchase,-,NSE,-


### Create another table with the following columns:

In [8]:
# Group the DataFrame by 'symbol' and aggregate data
df1 = df.groupby(['symbol']).agg({
    'secVal': 'sum',       # Sum the 'secVal' (total value of securities)
    'secAcq': 'sum',       # Sum the 'secAcq' (total quantity of securities acquired)
    'acqfromDt': 'max',     # Get the most recent acquisition date ('acqfromDt')
    'company': 'first'
}).reset_index()



### Calculates the average buy value per unit by dividing the total value by the total quantity and filters where total value is greater than 100 million

In [9]:

# Calculate the average buy value per unit by dividing total value by total quantity, rounded to 2 decimal places
df1['BuyValue'] = round(df1['secVal'] / df1['secAcq'], 2)

# Filter the grouped DataFrame to include only rows where the total security value ('secVal') is greater than 100 million
df1 = df1[df1['secVal'] > 100000000]

# Sort the DataFrame by the most recent acquisition date ('acqfromDt') in descending order
df1 = df1.sort_values(by='acqfromDt', ascending=False)

# Display the final DataFrame
df1


,symbol,secVal,secAcq,acqfromDt,company,BuyValue
79,STEL,162105118,387017.0,2024-08-12,Stel Holdings Limited,418.86
2,ADANIPOWER,8589300497,12300000.0,2024-08-07,Adani Power Limited,698.32
7,BAJAJFINSV,373950000,225000.0,2024-07-30,Bajaj Finserv Limited,1662.00
80,TIDEWATER,729393383,351000.0,2024-06-28,Tide Water Oil Company (India) Limited,2078.04
10,CANTABIL,148437242,679000.0,2024-06-27,Cantabil Retail India Limited,218.61
78,SNOWMAN,138464361,1987081.0,2024-06-27,Snowman Logistics Limited,69.68
16,DEEPAKNTR,365645422,146000.0,2024-06-26,Deepak Nitrite Limited,2504.42
58,PAISALO,588972464,8106075.0,2024-06-25,Paisalo Digital Limited,72.66
63,QUICKHEAL,2844791730,5817570.0,2024-06-18,Quick Heal Technologies Limited,489.00
8,BAJFINANCE,349762689,48000.0,2024-06-13,Bajaj Finance Limited,7286.72
